[Allen reference](https://demo.allennlp.org/reading-comprehension/bidaf-elmo)

In [1]:
# !pip install bert-extractive-summarizer --upgrade --force-reinstall
# !pip install --quiet git+https://github.com/huggingface/transformers.git@5c00918681d6b4027701eb46cea8f795da0d4064
# !pip install --quiet sentencepiece==0.1.95
# !pip install -U wn==0.0.22
# !pip install --quiet ipython-autotime
# !pip install monkeylearn
# !python3 -m spacy download en_core_web_sm
# !pip install flask-ngrok
# !pip install trafilatura
# !pip install allennlp==2.1.0 allennlp-models==2.1.0
# !pip install pymongo
# !pip install -U pywsd
# !pip install markupsafe==2.0.1
# !pip install spacy==3.4
# %load_ext autotime

In [2]:
import pandas as pd
hs_data = pd.read_csv('/content/haystack_model.csv')
hs_data.head()

,keyphrases,context,query,answer
0,"['egyptian civilization', 'egypt', 'nile river...","As Egyptian civilization grew more complex, pe...",What river fed Egyptian civilization for hundr...,The Nile River
1,"['egypt', 'underground tomb', 'priest', 'peopl...","As in many ancient societies, much of the know...",When did priests study the world to find ways ...,"ancient societies, much of the knowledge of Egypt"
2,"['egypt', 'underground tomb', 'priest', 'peopl...","As in many ancient societies, much of the know...","Where were veins of copper, iron, and bronze h...",desert mountains
3,"['egypt', 'underground tomb', 'priest', 'peopl...","As in many ancient societies, much of the know...",Who was the pharaoh?,a god
4,"['egypt', 'underground tomb', 'priest', 'peopl...","As in many ancient societies, much of the know...",What peninsula is east of Egypt?,Sinai Peninsula


In [3]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.rc
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-elmo.2021-02-11.tar.gz")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
downloading: 100%|##########| 374434792/374434792 [00:35<00:00, 10528248.09B/s]
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


In [6]:
al_data = dict()

al_data['keyphrases'] = hs_data['keyphrases']
al_data['context'] = hs_data['context']
al_data['query'] = hs_data['query']
al_data['answer'] = []

In [7]:
for i in range(len(al_data['query'])):
  ans = predictor.predict(
      passage=al_data['context'][i],
      question=al_data['query'][i]
  )['best_span_str']
  al_data['answer'].append(ans)

/usr/local/lib/python3.7/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [8]:
df = pd.DataFrame(al_data)
df.head()

,keyphrases,context,query,answer
0,"['egyptian civilization', 'egypt', 'nile river...","As Egyptian civilization grew more complex, pe...",What river fed Egyptian civilization for hundr...,Nile River
1,"['egypt', 'underground tomb', 'priest', 'peopl...","As in many ancient societies, much of the know...",When did priests study the world to find ways ...,As in many ancient societies
2,"['egypt', 'underground tomb', 'priest', 'peopl...","As in many ancient societies, much of the know...","Where were veins of copper, iron, and bronze h...",inside desert mountains
3,"['egypt', 'underground tomb', 'priest', 'peopl...","As in many ancient societies, much of the know...",Who was the pharaoh?,king of Egypt
4,"['egypt', 'underground tomb', 'priest', 'peopl...","As in many ancient societies, much of the know...",What peninsula is east of Egypt?,Sinai Peninsula


In [9]:
df.to_csv('/content/allen_model.csv', index=False, columns=df.columns)